In [ ]:


bank_system/
├── main.py                # For running the program
├── models/
│   ├── customer.py        # Customer class
│   ├── account.py         # Account, SavingsAccount, CurrentAccount classes
│   └── transaction.py     # Transaction class
├── bank.py                # BankSystem class
├── db/
│   └── db_connection.py   # MySQL connection + helper methods
└── test/
    └── test_bank.py       # Unit tests































import datetime

class Transaction:
    def __init__(self, txn_type, amount):
        self.txn_type = txn_type
        self.amount = amount
        self.timestamp = datetime.datetime.now()

    def __str__(self):
        return f"{self.timestamp} | {self.txn_type} | Rs.{self.amount}"

class Customer:
    def __init__(self, customer_id, name, address, phone):
        self.customer_id = customer_id
        self.name = name
        self.address = address
        self.phone = phone
        self.accounts = []

    def add_account(self, account):
        self.accounts.append(account)

    def get_details(self):
        return f"ID: {self.customer_id}, Name: {self.name},Phone: {self.phone}, Accounts: {len(self.accounts)}"


class Account:
    def __init__(self, account_number, balance=0):
        self.account_number = account_number
        self.balance = balance
        self.transactions = []

    def deposit(self, amount):
        self.balance += amount
        self.transactions.append(Transaction("Deposit", amount))

    def withdraw(self, amount):
        if amount > self.balance:
            raise Exception("Insufficient balance")
        self.balance -= amount
        self.transactions.append(Transaction("Withdraw", amount))

    def get_balance(self):
        return self.balance

    def get_statement(self):
        return [str(txn) for txn in self.transactions]

class SavingsAccount(Account):
    def __init__(self, account_number, balance=0, interest_rate=0.03):
        super().__init__(account_number, balance)
        self.interest_rate = interest_rate

    def apply_interest(self):
        interest = self.balance * self.interest_rate
        self.deposit(interest)

class CurrentAccount(Account):
    def __init__(self, account_number, balance=0, overdraft_limit=10000):
        super().__init__(account_number, balance)
        self.overdraft_limit = overdraft_limit

    def withdraw(self, amount):
        if amount > self.balance + self.overdraft_limit:
            raise Exception("Overdraft limit exceeded")
        self.balance -= amount
        self.transactions.append(Transaction("Withdraw", amount))

class BankSystem:
    def __init__(self):
        self.customers = []
        self.customer_id_seq = 1000
        self.account_number_seq = 5000

    def register_customer(self, name, address, phone):
        customer = Customer(self.customer_id_seq, name, address, phone)
        self.customers.append(customer)
        self.customer_id_seq += 1
        return customer

    def open_account(self, customer, account_type, initial_deposit=0):
        if account_type == "savings":
            account = SavingsAccount(self.account_number_seq, initial_deposit)
        elif account_type == "current":
            account = CurrentAccount(self.account_number_seq, initial_deposit)
        else:
            raise ValueError("Invalid account type")

        customer.add_account(account)
        self.account_number_seq += 1
        return account

    def find_customer_by_id(self, customer_id):
        for cust in self.customers:
            if cust.customer_id == customer_id:
                return cust
        return None

    def generate_report(self):
        print("Customer Report")
        print("-" * 40)
        for customer in self.customers:
            print(customer.get_details())

bank = BankSystem()

# Register a customer
cust1 = bank.register_customer("Nizamuddin", "Panipat", "8840193653")
cust2 = bank.register_customer("Ayesha Khan", "Delhi", "9900123456")

# Open accounts
acc1 = bank.open_account(cust1, "savings", 10000)
acc2 = bank.open_account(cust2, "current", 15000)

# Transactions
acc1.deposit(2000)
acc1.withdraw(1500)
acc2.withdraw(3000)

# Apply interest
acc1.apply_interest()

# Print mini statement
print("\nMini Statement for John Doe:")
for txn in acc1.get_statement():
    print(txn)

# Admin Report
print("\n--- Admin Report ---")
bank.generate_report()



Mini Statement for John Doe:
2025-07-25 14:40:15.407672 | Deposit | Rs.2000
2025-07-25 14:40:15.407672 | Withdraw | Rs.1500
2025-07-25 14:40:15.407672 | Deposit | Rs.315.0

--- Admin Report ---
Customer Report
----------------------------------------
ID: 1000, Name: Nizamuddin,Phone: 8840193653, Accounts: 1
ID: 1001, Name: Ayesha Khan,Phone: 9900123456, Accounts: 1


In [ ]:
#Unit Testing
import unittest

class TestBankSystem(unittest.TestCase):
    def test_deposit(self):
        acc = SavingsAccount(101, 1000)
        acc.deposit(500)
        self.assertEqual(acc.get_balance(), 1500)

    def test_withdraw(self):
        acc = SavingsAccount(101, 1000)
        acc.withdraw(200)
        self.assertEqual(acc.get_balance(), 800)

if __name__ == '__main__':
    unittest.main()
